Population Receptive Field Mapping properties\nin two groups, healthy controls and glaucoma patients

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cortex.freesurfer
import sys
import os
import seaborn as sns
import csv
#import scipy.stats as st
import pandas as pd
opj = os.path.join
import glob

In [2]:
# Directories 
main_path = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
trendline_output = os.path.join(f"/Users/federicacardillo/Desktop/EGRET-AAA/pRFmapping", 'trendlines')
os.makedirs(trendline_output, exist_ok=True)
group_output = os.path.join(f"/Users/federicacardillo/Desktop/EGRET-AAA/pRFmapping", 'group')
os.makedirs(group_output, exist_ok=True)

In [3]:
# Task settings 
rois_list = np.array([['V1', 'V2', 'V3', 'V4', 'LO'], [1, 2, 3, 4, 5]])
visual_areas = ['V1', 'V2', 'V3', 'V4', 'LO']
project = 'PROJECT_EGRET-AAA'
denoising = 'nordic'
atlas = 'manual'
tasks = ['RET', 'RET2']
min_r2 = 0.1
min_ecc = 0.1
max_ecc = 6.5
min_size = 0.01
ecc_bound = (0, 7)

# Analysis settings
confidence = 0.95
n_samples = 1000
offset = True 

# Dataset settings
#glaucoma = ['sub-02', 'sub-04']
#controls = ['sub-21', 'sub-26']
glaucoma = ['sub-02', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08','sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-48'] # Missing 08 and 19
controls = ['sub-21', 'sub-22', 'sub-23', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38', 'sub-39', 'sub-40', 'sub-41', 'sub-42','sub-43', 'sub-44','sub-45','sub-46']
subjects = glaucoma + controls
subjects = [d for d in os.listdir(os.path.join(main_path, 'pRFM')) if os.path.isdir(os.path.join(os.path.join(main_path, 'pRFM'), d)) and d in subjects]

In [4]:
oct_averageeye = {
    'sub-02': 73, 'sub-04': 57, 'sub-05': 59, 'sub-06': 72, 'sub-07': 86.5,
    'sub-08': 71, 'sub-09': 65, 'sub-10': 64.5, 'sub-11': 59.5, 'sub-12': 56,
    'sub-13': 63, 'sub-14': 73.5, 'sub-15': 74.5, 'sub-16': 65.5, 'sub-17': 70.5,
    'sub-18': 71, 'sub-19': 55, 'sub-20': 64, 'sub-21': 72.5, 'sub-22': 89,
    'sub-23': 68, 'sub-25': 105, 'sub-26': 87, 'sub-27': 100.5, 'sub-28': 69,
    'sub-29': 82.5, 'sub-30': 100, 'sub-31': 89.5, 'sub-32': 80.5, 'sub-33': 97.5,
    'sub-34': 100.5, 'sub-35': 100, 'sub-36': 103, 'sub-37': 95.5, 'sub-38': 100,
    'sub-39': 104.5, 'sub-40': 101, 'sub-41': 101, 'sub-42': 102, 'sub-43': 110.5,
    'sub-44': 105, 'sub-45': 100.5, 'sub-46': 94.5}
hfa_averageeye = {
    'sub-02': -15.83, 'sub-04': -20.76, 'sub-05': -9.25, 'sub-06': 0.62, 'sub-07': -2.88,
    'sub-08': -6.1, 'sub-09': -4.35, 'sub-10': -11.54, 'sub-11': -2.08, 'sub-12': -8.68,
    'sub-13': -9.76, 'sub-14': -8.06, 'sub-15': -1.9, 'sub-16': -1.68, 'sub-17': -5.46,
    'sub-18': -12.41, 'sub-19': -7.78, 'sub-20': -15.24, 'sub-21': -1.94, 'sub-22': -0.91,
    'sub-23': -7.54, 'sub-25': -0.38, 'sub-26': -0.38, 'sub-27': 1.42, 'sub-28': 0.29,
    'sub-29': -0.14, 'sub-30': 0.19, 'sub-31': 0.37, 'sub-32': 1.64, 'sub-33': -0.12,
    'sub-34': -2.16, 'sub-35': 0.07, 'sub-36': 0.84, 'sub-37': -2.01, 'sub-38': -0.39,
    'sub-39': 0.5, 'sub-40': 0.9, 'sub-41': -1.01, 'sub-42': 1.17, 'sub-43': -1.08,
    'sub-44': 1.23, 'sub-45': -0.28, 'sub-46': 0.7}
hfa_besteye = {
    'sub-02': -1.12, 'sub-04': -18.43, 'sub-05': -7.5, 'sub-06': 0.95, 'sub-07': -2.1,
    'sub-08': -0.09, 'sub-09': -2.56, 'sub-10': -8.37, 'sub-11': -2.05, 'sub-12': -4.7,
    'sub-13': -4.24, 'sub-14': -3.52, 'sub-15': 0.52, 'sub-16': -1.1, 'sub-17': -4.56,
    'sub-18': -0.92, 'sub-19': -6.11, 'sub-20': -2.36, 'sub-21': 0.38, 'sub-22': 0.24,
    'sub-23': -2.67, 'sub-25': -0.11, 'sub-26': 0.71, 'sub-27': 1.75, 'sub-28': 0.6,
    'sub-29': 0.2, 'sub-30': 0.97, 'sub-31': 0.51, 'sub-32': 1.73, 'sub-33': 0.26,
    'sub-34': -1.58, 'sub-35': 0.45, 'sub-36': 1.25, 'sub-37': -0.87, 'sub-38': 0.84,
    'sub-39': 1.35, 'sub-40': 1.69, 'sub-41': -0.89, 'sub-42': 1.76, 'sub-43': -0.95,
    'sub-44': 1.91, 'sub-45': -0.22, 'sub-46': 0.97}
oct_besteye = {
    'sub-02': 101, 'sub-04': 58, 'sub-05': 64, 'sub-06': 73, 'sub-07': 91,
    'sub-08': 78, 'sub-09': 76, 'sub-10': 66, 'sub-11': 60, 'sub-12': 63,
    'sub-13': 66, 'sub-14': 75, 'sub-15': 86, 'sub-16': 71, 'sub-17': 72,
    'sub-18': 86, 'sub-19': 60, 'sub-20': 71, 'sub-21': 77, 'sub-22': 95,
    'sub-23': 75, 'sub-25': 108, 'sub-26': 89, 'sub-27': 101, 'sub-28': 93,
    'sub-29': 86, 'sub-30': 101, 'sub-31': 91, 'sub-32': 82, 'sub-33': 99,
    'sub-34': 101, 'sub-35': 104, 'sub-36': 112, 'sub-37': 105, 'sub-38': 101,
    'sub-39': 105, 'sub-40': 102, 'sub-41': 101, 'sub-42': 104, 'sub-43': 111,
    'sub-44': 108, 'sub-45': 103, 'sub-46': 96}
oct_worsteye = {
    'sub-02': 79, 'sub-04': 56, 'sub-05': 54, 'sub-06': 71, 'sub-07': 82,
    'sub-08': 64, 'sub-09': 54, 'sub-10': 63, 'sub-11': 59, 'sub-12': 49,
    'sub-13': 60, 'sub-14': 72, 'sub-15': 63, 'sub-16': 60, 'sub-17': 69,
    'sub-18': 56, 'sub-19': 50, 'sub-20': 57, 'sub-21': 68, 'sub-22': 83,
    'sub-23': 61, 'sub-25': 102, 'sub-26': 85, 'sub-27': 100, 'sub-28': 45,
    'sub-29': 79, 'sub-30': 99, 'sub-31': 88, 'sub-32': 79, 'sub-33': 96,
    'sub-34': 100, 'sub-35': 96, 'sub-36': 94, 'sub-37': 86, 'sub-38': 99,
    'sub-39': 104, 'sub-40': 100, 'sub-41': 101, 'sub-42': 100, 'sub-43': 110,
    'sub-44': 102, 'sub-45': 98, 'sub-46': 93}
hfa_worsteye = {
    'sub-02': -30.54, 'sub-04': -23.08, 'sub-05': -11, 'sub-06': 0.3, 'sub-07': -3.67,
    'sub-08': -12.11, 'sub-09': -6.14, 'sub-10': -14.72, 'sub-11': -2.12, 'sub-12': -12.66,
    'sub-13': -15.29, 'sub-14': -12.6, 'sub-15': -4.32, 'sub-16': -2.25, 'sub-17': -6.36,
    'sub-18': -23.9, 'sub-19': -9.45, 'sub-20': -28.11, 'sub-21': -4.25, 'sub-22': -2.06,
    'sub-23': -12.4, 'sub-25': -0.65, 'sub-26': -1.47, 'sub-27': 1.08, 'sub-28': -0.02,
    'sub-29': -0.48, 'sub-30': -0.59, 'sub-31': 0.23, 'sub-32': 1.55, 'sub-33': -0.49,
    'sub-34': -2.73, 'sub-35': -0.31, 'sub-36': 0.44, 'sub-37': -3.15, 'sub-38': -1.62,
    'sub-39': -0.36, 'sub-40': 0.12, 'sub-41': -1.12, 'sub-42': 0.58, 'sub-43': -1.21,
    'sub-44': 0.55, 'sub-45': -0.34, 'sub-46': 0.43}

In [5]:
class PRFModel:
    '''Class representing pRF model parameters '''
    def __init__(self, r2, size, ecc, angle):
        """
        Initialize the PRFModel instance with the following parameters:
            
        Parameters:
        r2 (float): Model fit value (R-squared). Represents how well the model explains the variance in the observed data. 
                    A higher value indicates a better fit.
        size (float): Size of the pRF (population receptive field), which describes the spatial extent or diameter of the receptive field.
        ecc (float): Eccentricity. Represents the distance of the pRF from the center of the visual field. Higher values mean the receptive field 
                    is located farther from the center (fovea).
        angle (float): Polar angle. Specifies the direction of the receptive field from the center, often in degrees or radians.
        """
        self.r2 = r2            # Model fit - represents how well the model explains data variance
        self.size = size        # Size of the pRF (population receptive field)
        self.ecc = ecc          # Eccentricity - distance of the pRF from the center
        self.angle = angle      # Polar angle - position angle of the pRF

def load_pickle_file(filepath):
    '''Function to load a pickle file given a file path'''
    # Check if the given file path points to an existing file
    if not os.path.isfile(filepath):
        # If the file does not exist, raise a FileNotFoundError
        #raise FileNotFoundError(f"File not found: {filepath}")
        print(f"File not found: {filepath}")            
        return None
    # Open the file in 'rb' mode (read binary) since pickle files contain serialized binary data
    with open(filepath, 'rb') as file: 
        # Deserializes the contents of the binary file and returns the Python object.
        return pickle.load(file)

def load_prf_params(subjectid, main_path, atlas, denoising, task):
    '''Function to load pRF parameters for a given subjects and other details'''
    # Construct the file path to the pRF parameters file based on input parameters
    # The path points to a pickle file that contains the pRF model information
    filepath = os.path.join(main_path, f'pRFM/{subjectid}/ses-02/{denoising}/model-{atlas}-nelder-mead-GM_desc-prf_params_{task}.pkl') 
    # Load the pickle file using the load_pickle_file function defined previously
    pkl_data = load_pickle_file(filepath)           
    if pkl_data is None:
        return None, None            
    # Extract the pRF model parameters using the 'iterative_search_params' key from the loaded data
    prf_params = pkl_data['model'].iterative_search_params     
    # Extract the indices of the voxels that fall within the region of interest (ROI) 
    # This is done by finding where the 'rois_mask' is equal to 1 (indicating the voxel is inside the ROI)
    prf_voxels = np.where(pkl_data['rois_mask'] == 1)[0]      
    # Return both the extracted pRF model parameters and the indices of the voxels within the ROI
    return prf_params, prf_voxels

def load_labels(subjectid, main_path, atlas):
    '''Function to load FreeSurfer labels for the specified subject'''
    # Define the path to the FreeSurfer directory within the main project directory
    fs_dir = os.path.join(main_path, 'freesurfer') 
    try:
        # Load label indices for visual areas and eccentricity regions:
        # Load visual area labels (V1, V2, V3, etc.) using Benson's visual area label ('benson14_varea-0001')
        idx_rois4, idx_vls4 = cortex.freesurfer.get_label(subjectid, label='benson14_varea-0001', fs_dir=fs_dir, hemisphere=('lh', 'rh'))
        # Load eccentricity labels using Benson's eccentricity label ('benson14_eccen-0001')
        idx_rois1, idx_vls1 = cortex.freesurfer.get_label(subjectid, label='benson14_eccen-0001', fs_dir=fs_dir, hemisphere=('lh', 'rh'))
        # Check if 'manual' atlas is specified
        # If a manual atlas is requested, load the manual labels and update the corresponding indices
        if atlas == 'manual': 
            idx_rois_manual, idx_vls_manual = cortex.freesurfer.get_label(subjectid, label='manualdelin', fs_dir=fs_dir, hemisphere=('lh', 'rh'))
            # Update the values in idx_vls4 to use the manual labels wherever applicable
            idx_vls4[idx_rois_manual] = idx_vls_manual
    except Exception as e:
        # If there is an error loading any of the labels, raise a RuntimeError
        raise RuntimeError(f"Error loading labels for subject {subjectid}: {e}")
    # Return the loaded indices and values:
    # - idx_rois4, idx_vls4: Visual area indices and labels
    # - idx_vls1: Eccentricity labels
    return idx_rois4, idx_vls4, idx_vls1

def filter_prf_data(prf_params, prf_voxels):
    '''Filter and extract pRF data for the selected voxels'''
    # Construct and return a PRFModel object containing specific parameters of interest
    # - r2, size, eccentricity, and polar angle for the selected voxels.
    return PRFModel(
        r2=prf_params[:, 7],                                                # Extract R2 values from column 7 of prf_params.
        size=prf_params[:, 2],                                              # Extract pRF size values from column 2 of prf_params.
        # Eccentricity is calculated as the Euclidean distance from the center (0,0)
        # This is done using the Pythagorean theorem: ecc = sqrt(x^2 + y^2).
        ecc=np.sqrt(prf_params[:, 1]**2 + prf_params[:, 0]**2),             # Calculate eccentricity 
        # np.arctan2(y, x) computes the angle (in radians) of the vector formed by (x, y) relative to the positive x-axis.
        # The function np.rad2deg() converts the resulting angle to degrees.
        angle=np.rad2deg(np.arctan2(prf_params[:, 1], prf_params[:, 0])))

In [6]:
def bootstrap_weighted_avg(x, y, r2, n_samples, ecc_bound, roi_name, subject_num, offset, group_type=None):
    '''Function to create a bootstrapped weighted average plot'''
    # Set the Seaborn plotting style to 'white' for a clean background
    sns.set_style('white')
    # Assign color based on the group type or ROI name
    colors = {'V1': 'blue', 'V2': 'orange', 'V3': 'green', 'V4': 'pink', 'LO': 'red', 'glaucoma': 'red', 'control': 'blue'}
    # Get the color for the group or ROI; default to 'gray' if group/ROI is not in the dictionary
    group_color = colors.get(group_type, colors.get(roi_name, 'gray'))
    # Scatter plot of the x and y data, with transparency set by the R2 value
    plt.scatter(x, y, alpha=r2, marker='o', s=1, color=group_color)

    # Sort the data to fit trend lines properly 
    sorted_idx = np.argsort(x)
    x, y, r2 = x[sorted_idx], y[sorted_idx], r2[sorted_idx]
    # Generate evenly spaced x values for fitting trendlines within the specified eccentricity bounds
    x_fit = np.linspace(ecc_bound[0], ecc_bound[1], len(x))
    y_fit_samples = np.zeros((n_samples, len(x_fit)))

    # Bootstrapping process: create multiple trend lines based on resampled data
    for i in range(n_samples):
        # Randomly choose indices to create a bootstrap sample
        rnd_idx = np.random.choice(len(x), size=len(x), replace=True)
        # Resample the x, y, and r2 data using the random indices
        res_x, res_y, res_r2 = x[rnd_idx], y[rnd_idx], r2[rnd_idx]
        # Fit a linear model (degree 1 polynomial) to the resampled data, weighted by r2
        # model = np.polyfit(res_x, res_y, deg=1, w=res_r2)
        model = np.polyfit(res_x, res_y, deg=1) # not weight by VE 
        # Calculate the y-values for the fitted line and store in y_fit_samples
        if offset == True: 
            y_fit_samples[i, :] = model[0] * x_fit + model[1]
        else: 
            y_fit_samples[i, :] = model[0] * x_fit

    # Calculate the mean trend line and the confidence interval bounds from bootstrapped samples
    # MEDIAN TRENDLINE
    y_mean_fit = np.median(y_fit_samples, axis=0)
    # MEAN TREND LINE 
    # y_mean_fit = np.mean(y_fit_samples, axis=0)
    # y_mean_fit = model[0] * x_fit  
    lower_bounds = np.percentile(y_fit_samples, 2.5, axis=0) # 2.5th percentile for lower bound
    upper_bounds = np.percentile(y_fit_samples, 97.5, axis=0) # 97.5th percentile for upper bound
    
    # Fit the final model to the entire dataset to extract trendline parameters (slope and offset)
    final_model = np.polyfit(x, y, deg=1, w=r2)
    slope, offset = final_model

    # Plot the trend line based on the mean fit and add a label to the plot
    plt.plot(x_fit, y_mean_fit, ls='solid', linewidth=1.5, alpha=1, color=group_color, 
         label=f'{roi_name} (N = {len(r2)}): y = {np.polyfit(x, y, deg=1, w=r2)[0]:.4f}x + {np.polyfit(x, y, deg=1, w=r2)[1]:.4f}')
    # Plot the confidence interval as a shaded area around the trend line
    plt.fill_between(x_fit, lower_bounds, upper_bounds, color='gray', alpha=0.2)
    # Add title, labels, limits, and legend to the plot
    plt.title(f'{subject_num}: Trendline for {roi_name}', fontsize=18)
    plt.xlabel('Eccentricity', fontsize=16) # Label for the x-axis
    plt.ylabel('pRF Size', fontsize=16) # Label for the y-axis
    plt.ylim([0, 4]) # Set y-axis limits to maintain consistent scale
    plt.legend(loc='upper left') # Position the legend in the upper left
    plt.grid(True) # Display a grid on the plot
    plt.tight_layout() # Adjust the padding of the plot for a clean layout

    return x_fit, y_mean_fit, lower_bounds, upper_bounds

In [7]:
def trendlines(subject_num, prf_model, idx_vls4, prf_voxels, rois_list, trendline_output, n_samples, task, atlas, min_r2, min_ecc, max_ecc, min_size, ecc_bound, offset):
    # Load each subject corresponding to the groups and tasks 
    if task == "RET":
        group = 'Glaucoma' if subject_num in glaucoma else 'Controls'
    else:
        group = 'Glaucoma_MO' if subject_num in controls else 'Controls_SS'
    csv_path = os.path.join(trendline_output, f'{atlas}_{task}_{group}.csv'
    ) 
    csv_data = []  # To collect trendline data for CSV

    # Plotting the Trendlines for each subject
    fig, ax = plt.subplots(figsize=(12, 8))
    axes = [ax] 
    for i, roi_name in enumerate(rois_list[0]):
        ax = axes[0]
        roi_idx = np.where(roi_name == rois_list[0, :])[0]
        roi_vertices = np.array(np.where(idx_vls4 == int(rois_list[1, roi_idx])))[0]
        idx = np.in1d(prf_voxels, roi_vertices)
        filtered_params = {'ecc': prf_model.ecc[idx],'angle': prf_model.angle[idx],'size': prf_model.size[idx],'r2': prf_model.r2[idx]}
        valid_idx = (filtered_params['r2'] > min_r2) & (filtered_params['ecc'] > min_ecc ) & (filtered_params['ecc'] < max_ecc) & (filtered_params['size'] > min_size)
        ecc = filtered_params['ecc'][valid_idx]
        data = filtered_params['size'][valid_idx]
        r2_values = filtered_params['r2'][valid_idx]
        x_fit, y_mean_fit, lower_bounds, upper_bounds = bootstrap_weighted_avg(x=ecc, y=data, r2=r2_values, n_samples=n_samples, ecc_bound=ecc_bound, 
                                                                               roi_name=roi_name, subject_num=subject_num, offset = True)
        # Fit final model to get slope/offset
        final_model = np.polyfit(ecc, data, deg=1, w=r2_values)
        slope, offset = final_model
        
        # Collect ophthalmological test results 
        hfa_value = hfa_averageeye.get(subject_num, 'N/A')  
        oct_value = oct_averageeye.get(subject_num, 'N/A') 
        hfa_worst = hfa_worsteye.get(subject_num, 'N/A')
        oct_worst  = oct_worsteye.get(subject_num, 'N/A')
        hfa_best = hfa_besteye.get(subject_num, 'N/A')
        oct_best  = oct_besteye.get(subject_num, 'N/A')
        csv_data.append([subject_num, roi_name, offset, slope, lower_bounds[0],upper_bounds[0],group, hfa_value, oct_value, hfa_worst, oct_worst,hfa_best,oct_best])              
            
    plot_path = os.path.join(trendline_output, f'{subject_num}_{task}.png')
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close(fig)

    # Save the trendline data to a CSV file
    if not os.path.exists(csv_path):
        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Subject Number', 'Visual Area', 'Offset', 'Slope','Confidence Interval Lower Bound', 'Confidence Interval Upper Bound',
                'Group', 'HFA Average', 'OCT Average', 'HFA Worst','OCT Worst', 'HFA Best', 'OCT Best'])
            writer.writerows(csv_data)
    else:
        with open(csv_path, 'a', newline='') as f:
            csv.writer(f).writerows(csv_data)

In [8]:
def group_comparison(data, main_path, visual_areas, groups, atlas='manual', n_samples=1000, confidence=0.95, task='RET'):
    with PdfPages(os.path.join(group_output, f'groupcomparison_{atlas}.pdf')) as pdf_pages:
        # Plotting the trendlines for each visual area
        for visual_area in visual_areas:
            plt.figure(figsize=(8, 8))
            for group_label, group_color in groups.items():
                group_df = data[(data['Group'] == group_label) & (data['Visual Area'] == visual_area)]
                # Store the data for each group
                all_eccentricities = []
                all_sizes = []
                all_r2_values = []

                for index, row_data in group_df.iterrows():
                    subject_id = row_data['Subject Number']
                    prf_params_data, prf_voxel_indices = load_prf_params(subject_id, main_path, atlas, 'nordic', task)
                    label_indices, visual_area_labels, ecc_labels = load_labels(subject_id, main_path, atlas)
                    prf_model_data = filter_prf_data(prf_params_data, prf_voxel_indices)

                    visual_area_labels = np.array(visual_area_labels)
                    roi_index = int(rois_list[1, rois_list[0] == visual_area][0])
                    visual_area_voxels = np.where(visual_area_labels == roi_index)[0]
                    valid_voxel_mask = np.in1d(prf_voxel_indices, visual_area_voxels)

                    size_values = prf_model_data.size[valid_voxel_mask]
                    eccentricity_values = prf_model_data.ecc[valid_voxel_mask]
                    r2_values = prf_model_data.r2[valid_voxel_mask]
                    valid_data_mask = (r2_values > 0.3) & (eccentricity_values > 0) & (eccentricity_values < 6.5) & (size_values > 0.11)
                    all_eccentricities.extend(eccentricity_values[valid_data_mask])
                    all_sizes.extend(size_values[valid_data_mask])
                    all_r2_values.extend(r2_values[valid_data_mask])

                all_eccentricities = np.array(all_eccentricities)
                all_sizes = np.array(all_sizes)
                all_r2_values = np.array(all_r2_values)

                x_values, mean_trendline, lower_ci, upper_ci = bootstrap_weighted_avg(x=all_eccentricities, y=all_sizes, r2=all_r2_values, n_samples=n_samples, ecc_bound=(0, 7), roi_name=visual_area, subject_num=f'{group_label} group', group_type=group_label, offset = True)
                plt.plot(x_values, mean_trendline, color=group_color, linewidth=2, label=group_label)

            plt.ylim(0, 3)
            plt.xlabel('Eccentricity', fontsize=20)
            plt.ylabel('Population Receptive Field Size', fontsize=20)
            plt.title(f'{visual_area}', fontsize=20)
            plt.grid(True)
            plt.tight_layout()
            pdf_pages.savefig()
            plt.close()

In [ ]:
# MAIN SCRIPT
#if __name__ == "__main__":
for subjectid in subjects:
    for task in tasks: 
        
        # Trendlines for each subject
        prf_params, prf_voxels = load_prf_params(subjectid, main_path, atlas, denoising, task)
        if prf_params is None or prf_voxels is None:
            print(f"Skipping {subjectid}, {task} due to missing data.")
            continue
        idx_rois4, idx_vls4, _ = load_labels(subjectid, main_path, atlas)
        prf_model = filter_prf_data(prf_params, prf_voxels)
        trendlines(subjectid, prf_model, idx_vls4, prf_voxels, rois_list, trendline_output, n_samples, task, atlas,
            min_r2, min_ecc, max_ecc, min_size, ecc_bound, offset)

        #data = pd.read_csv(csv_path)
        #data['Group'] = np.where(data['Subject Number'].isin(glaucoma), 'Primary Open-Angle Glaucoma MO', np.where(data['Subject Number'].isin(controls), 'Healthy Controls SS', ''))
        #groups = {'Primary Open-Angle Glaucoma MO': 'red', 'Healthy Control SS': 'blue'}
        # group_comparison(data, main_path, visual_areas, groups, atlas=atlas, task=task, offset=offset)

looking for ['/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/lh.benson14_varea-0001.label', '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/rh.benson14_varea-0001.label']
looking for ['/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/lh.benson14_eccen-0001.label', '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/rh.benson14_eccen-0001.label']
looking for ['/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/lh.manualdelin.label', '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/rh.manualdelin.label']
looking for ['/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/freesurfer/sub-02/label/lh.benson14_varea-0001.label', '/Volumes/Fe